In [1]:
import json
import re
import pandas as pd
from typing import Dict, List, Tuple, Union
from nbconvert.filters import ipython2python

In [2]:
with open("1_Using-Jupyter-Lab.ipynb") as test_notebook_file:
    test_notebook = json.load(test_notebook_file)["cells"]
pd.DataFrame(test_notebook).tail()

,cell_type,metadata,source,execution_count,outputs
27,code,"{'tags': ['raises-exception', 'flake8-noqa-cel...",[import not_a_valid_module],11.0,"[{'ename': 'ModuleNotFoundError', 'evalue': 'N..."
28,code,"{'tags': ['flake8-noqa-cell-E402-F401', 'flake...","[for i in range(2):\n, print(i)]",12.0,"[{'name': 'stdout', 'output_type': 'stream', '..."
29,code,{},"[%%python\n, print(""foo"")]",14.0,"[{'name': 'stdout', 'output_type': 'stream', '..."
30,code,{'tags': ['flake8-noqa']},"[%ls\n, import __hello__]",17.0,"[{'name': 'stdout', 'output_type': 'stream', '..."
31,code,{},[],NaN,[]


In [3]:
def is_cell_magic(source_line):
    return source_line.startswith("%%")

for index, cell in list(enumerate(test_notebook))[::-1]:
    if cell["cell_type"] != "code":
        test_notebook.pop(index)
    elif not cell["source"]:
        test_notebook.pop(index)
    elif any(map(is_cell_magic, cell["source"])):
        test_notebook.pop(index)
    else:
        for source_line_index, source_line in list(enumerate(cell["source"]))[::-1]:
            if source_line.startswith(("%", "?", "!")):
                cell["source"].pop(source_line_index)
        if not cell["source"]:
            test_notebook.pop(index)
            

pd.DataFrame(test_notebook)

,cell_type,execution_count,metadata,outputs,source
0,code,9,{'tags': ['flake8-noqa']},[{'data': {'text/plain': ['\x1b[1;31mInit sign...,[map?\n]
1,code,10,{'tags': ['flake8-noqa']},[{'data': {'text/plain': ['\x1b[1;31mInit sign...,[map?]
2,code,11,"{'tags': ['raises-exception', 'flake8-noqa-cel...","[{'ename': 'ModuleNotFoundError', 'evalue': 'N...",[import not_a_valid_module]
3,code,12,"{'tags': ['flake8-noqa-cell-E402-F401', 'flake...","[{'name': 'stdout', 'output_type': 'stream', '...","[for i in range(2):\n, print(i)]"
4,code,17,{'tags': ['flake8-noqa']},"[{'name': 'stdout', 'output_type': 'stream', '...",[import __hello__]


## Input Notebook

In [28]:
def ignore_cell(notebook_cell):
    if not notebook_cell["source"]:
        return True
    elif notebook_cell["cell_type"] != "code":
        return True

    
def get_clean_notebook(notebook_path):
    with open(notebook_path) as notebook_file:
        notebook_cells = json.load(notebook_file)["cells"]        
    for index, cell in list(enumerate(notebook_cells))[::-1]:
        if ignore_cell(cell):
            notebook_cells.pop(index)
        else:
            cell_source = list(enumerate(cell["source"]))[::-1]
            for source_index, source_line in cell_source:
                cell["source"][source_index] = ipython2python(source_line)
    return notebook_cells
    
test_notebook = get_clean_notebook("1_Using-Jupyter-Lab.ipynb")
pd.DataFrame(test_notebook)

,cell_type,execution_count,metadata,outputs,source
0,code,1,{'tags': ['nbval-ignore-output']},[{'data': {'application/json': {'cell': {'!': ...,"[get_ipython().run_line_magic('lsmagic', '')\n]"
1,code,2,"{'tags': ['nbval-ignore-output', 'flake8-noqa-...","[{'name': 'stdout', 'output_type': 'stream', '...","[get_ipython().run_cell_magic('bash', '', '')\..."
2,code,3,{'tags': ['nbval-ignore-output']},"[{'name': 'stdout', 'output_type': 'stream', '...",[get_ipython().system('ls -la')\n]
3,code,4,"{'tags': ['nbval-ignore-output', 'flake8-noqa-...",[{'data': {'text/plain': ['\x1b[1;31mDocstring...,"[get_ipython().run_line_magic('pinfo', '%%bash..."
4,code,5,"{'tags': ['nbval-ignore-output', 'flake8-noqa-...",[{'data': {'text/plain': ['\x1b[1;31mDocstring...,"[get_ipython().run_line_magic('pinfo', '%%bash..."
5,code,6,{'tags': ['flake8-noqa-cell']},[{'data': {'text/plain': ['\x1b[1;31mInit sign...,"[get_ipython().run_line_magic('pinfo', 'map')\n]"
6,code,7,{'tags': ['flake8-noqa-cell']},[{'data': {'text/plain': ['\x1b[1;31mInit sign...,"[get_ipython().run_line_magic('pinfo', 'map')\n]"
7,code,8,"{'tags': ['raises-exception', 'flake8-noqa-cel...","[{'ename': 'ModuleNotFoundError', 'evalue': 'N...",[import not_a_valid_module\n]
8,code,9,"{'tags': ['flake8-noqa-cell-E402-F401', 'flake...","[{'name': 'stdout', 'output_type': 'stream', '...","[for i in range(2):\n, print(i)\n]"
9,code,10,{},"[{'name': 'stdout', 'output_type': 'stream', '...","[get_ipython().run_cell_magic('python', '', ''..."


In [5]:
def generate_input_name(notebook_path, input_nr):
    return f"{notebook_path}#In[{input_nr}]"
generate_input_name("1_Using-Jupyter-Lab.ipynb", 7)

'1_Using-Jupyter-Lab.ipynb#In[7]'

In [6]:
def strip_newline(souce_line):
    return souce_line.rstrip("\n")
def add_newline(souce_line):
    return f"{souce_line}\n"
striped_source = list(map(strip_newline, test_notebook[1]["source"]))
print("striped_source:\n","".join(striped_source))
reassembled_source = map(add_newline, striped_source)
print("reassembled_source:\n","".join(reassembled_source))

striped_source:
 get_ipython().run_cell_magic('bash', '', '')ls -la
reassembled_source:
 get_ipython().run_cell_magic('bash', '', '')
ls -la



In [7]:
test_notebook[1]["metadata"].get("tags", None)

['nbval-ignore-output', 'flake8-noqa']

## Flake8 tags to use

### cell tags
Apply rules to all lines of the cell

pattern: ``flake8-noqa-cell-<rule1>-<rule2>``
    
example: flake8-noqa-cell-E402-F401

#### ignore tag
Whole cell will be ignored

pattern: ``flake8-noqa-cell``

### line tags
Apply rules to line of the cell, with line_nr 

pattern: ``flake8-noqa-line-<line_nr>-<rule1>-<rule2>``
    
example: flake8-noqa-line-1-E402-F401

#### ignore tag
Line will be ignored

pattern: ``flake8-noqa-line-<line_nr>``

example: flake8-noqa-line-1

### Regex
``^flake8-noqa-(cell-(?P<cell_rules>(\w+\d+-?)+)|line-(?P<line_nr>\d+)-(?P<line_rules>(\w+\d+-?)+))$|^(?P<ignore_cell>flake8-noqa-cell)$|^flake8-noqa-line-(?P<ignore_line_nr>\d+)$``

testcase:
```
flake8-noqa-cell-E402-F401
flake8-noqa-cell
flake8-noqa-line-1-E402-F401
flake8-noqa-line-1
flake8-noqa-line-foo-E402-F401
```

In [21]:
def extract_flake8_tags(notebook_cell: Dict):
    input_nr = notebook_cell["execution_count"]
    flake8_tags = []
    for tag in notebook_cell["metadata"].get("tags", []):
        if tag.startswith("flake8-noqa"):
            flake8_tags.append(tag)
    return {"input_nr": input_nr, "flake8_tags": flake8_tags}
extract_flake8_tags(test_notebook[0])

{'input_nr': 4, 'flake8_tags': []}

In [9]:
extract_flake8_tags(test_notebook[1])

nbval-ignore-output
flake8-noqa


{'input_nr': 5, 'flake8_tags': ['flake8-noqa']}

In [16]:
flake8_tag_pattern = (r"^flake8-noqa-(cell-(?P<cell_rules>(\w+\d+-?)+)"
                      r"|line-(?P<line_nr>\d+)-(?P<line_rules>(\w+\d+-?)+))$"
                      r"|^(?P<ignore_cell>flake8-noqa-cell)$"
                      r"|^flake8-noqa-line-(?P<ignore_line_nr>\d+)$")
flake8_tag_regex = re.compile(flake8_tag_pattern)
match1 = re.match(flake8_tag_regex, 'flake8-noqa-cell-E402-F401')
print('matching: "flake8-noqa-cell-E402-F401"')
print("cell_rules", match1.group("cell_rules"))
print("line_nr", match1.group("line_nr"))
print("line_rules", match1.group("line_rules"))
match2 = re.match(flake8_tag_regex, 'flake8-noqa-line-1-E402-F401')
print('matching: "flake8-noqa-line-1-E402-F401"')
print("cell_rules", match2.group("cell_rules"))
print("line_nr", match2.group("line_nr"))
print("line_rules", match2.group("line_rules"))

matching: "flake8-noqa-cell-E402-F401"
cell_rules E402-F401
line_nr None
line_rules None
matching: "flake8-noqa-line-1-E402-F401"
cell_rules None
line_nr 1
line_rules E402-F401


## Tags to rules-dict

In [18]:
import warnings

class InvalidFlake8TagWarning(UserWarning):
    pass

def warn_wrong_tag_pattern(flake8_tag: str):
    warnings.warn("flake8-noqa-line/cell-tags should be of form "
                  "'flake8-noqa-cell-<rule1>-<rule2>'|'flake8-noqa-cell'/"
                  "'flake8-noqa-line-<line_nr>-<rule1>-<rule2>'|'flake8-noqa-line-<rule1>', "
                  f"you used: '{flake8_tag}'", InvalidFlake8TagWarning)

def flake8_tag_to_rules_dict(flake8_tag_regex: re.Pattern, flake8_tag: str) -> Dict[str, List]:
    match = re.match(flake8_tag_regex, flake8_tag)
    if match:
        if match.group("cell_rules"):
            cell_rules = match.group("cell_rules")
            cell_rules = cell_rules.split("-")
            return {"cell": cell_rules}
        elif match.group("ignore_cell"):
            return {"cell": ["noqa"]}
        elif match.group("line_nr") and match.group("line_rules") :
            line_nr = str(match.group("line_nr"))
            line_rules = match.group("line_rules")
            line_rules = line_rules.split("-")
            return {line_nr: line_rules}
        elif match.group("ignore_line_nr"):
            line_nr = str(match.group("ignore_line_nr"))
            return {line_nr: ["noqa"]}
    warn_wrong_tag_pattern(flake8_tag)
    return {}
    
print(flake8_tag_to_rules_dict(flake8_tag_regex, 'flake8-noqa-cell-E402-F401'))
print(flake8_tag_to_rules_dict(flake8_tag_regex, 'flake8-noqa-cell'))
print(flake8_tag_to_rules_dict(flake8_tag_regex, 'flake8-noqa-line-1-E402-F401'))
print(flake8_tag_to_rules_dict(flake8_tag_regex, 'flake8-noqa-line-1'))
print(flake8_tag_to_rules_dict(flake8_tag_regex, 'flake8-noqa-line-foo-E402-F401'))

{'cell': ['E402', 'F401']}
{'cell': ['noqa']}
{'1': ['E402', 'F401']}
{'1': ['noqa']}
{}


C:\Anaconda3\lib\site-packages\ipykernel_launcher.py:10: InvalidFlake8TagWarning: flake8-noqa-line/cell-tags should be of form 'flake8-noqa-cell-<rule1>-<rule2>'|'flake8-noqa-cell'/'flake8-noqa-line-<line_nr>-<rule1>-<rule2>'|'flake8-noqa-line-<rule1>', you used: 'flake8-noqa-line-foo-E402-F401'
  # Remove the CWD from sys.path while we load stuff.


## Total rules dict

In [34]:
def update_rules_dict(total_rules_dict: Dict[str, List], new_rules_dict: Dict[str, List]) -> Dict[str, List]:
    for key, new_rules in new_rules_dict.items():
        old_rules = total_rules_dict.get(key, [])
        if 'noqa' in old_rules+new_rules:
            total_rules_dict[key] = ['noqa']
        else:
            total_rules_dict[key] = old_rules + new_rules
        

def get_flake8_rules_dict(notebook_cell: Dict) -> Tuple[int, Dict]:
    flake8_tags = extract_flake8_tags(notebook_cell)
    flake8_tag_pattern = (r"^flake8-noqa-(cell-(?P<cell_rules>(\w+\d+-?)+)"
                          r"|line-(?P<line_nr>\d+)-(?P<line_rules>(\w+\d+-?)+))$"
                          r"|^(?P<ignore_cell>flake8-noqa-cell)$"
                          r"|^flake8-noqa-line-(?P<ignore_line_nr>\d+)$")
    flake8_tag_regex = re.compile(flake8_tag_pattern)
    total_rules_dict = {}
    for flake8_tag in flake8_tags["flake8_tags"]:
        new_rules_dict = flake8_tag_to_rules_dict(flake8_tag_regex, flake8_tag)
        update_rules_dict(total_rules_dict, new_rules_dict)
    return flake8_tags["input_nr"], total_rules_dict
        

In [35]:
get_flake8_rules_dict(test_notebook[7])

(8, {'cell': ['E402', 'F401']})

In [36]:
get_flake8_rules_dict(test_notebook[8])

(9, {'cell': ['noqa'], '1': ['E402', 'F401', 'W391']})

In [14]:
from nbconvert.filters import ipython2python
ipython2python("%%bash\necho help\necho help2")

"get_ipython().run_cell_magic('bash', '', 'echo help\\necho help2')\n"

In [15]:
ipython2python("%%bash\necho help\necho help2\n")

"get_ipython().run_cell_magic('bash', '', 'echo help\\necho help2')\n"

``^.+?\s*(?P<is_flake8_noqa>[#]\s*noqa\s*[:](\s*\w+\d+[,]?\s*)+)$``

```python
foo  # noqa: E402, Fasd401
foo  # noqa : E402, Fasd401 
"test  # noqa: E402, Fasd401"
```